TASK1
（由于生病等一些原因，task1没足够时间实现，只能简单梳理思路，希望这周末把全部代码补完 19-07-18 周四晚）

In [1]:
import numpy as np
import pandas as pd
import os
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

Part1
数据处理
1.取1/3训练集作为试验集，其余为训练集
2.导入数据并使用nltk对train.Phrase分词并提取词频率

In [2]:
train= pd.read_csv("xiyu/task1/input/train.tsv", sep="\t")
test = pd.read_csv("xiyu/task1/input/test.tsv", sep="\t")
print(train.iloc[0]['Phrase'],'Sentiment - ',train.iloc[0]['Sentiment'])

A series of escapades demonstrating the adage that what is good for the goose is also good for the gander , some of which occasionally amuses but none of which amounts to much of a story . Sentiment -  1


Part2
特征 
保留词频前50的单词，作为词向量。并利用词向量分别对训练集中的句子train.Phrase处理，得到新的一列‘V_Phrase’(1*50向量)，对train.Sentiment,使用softmax处理得到其对应的值

In [4]:
print(train.Phrase)

0         A series of escapades demonstrating the adage ...
1         A series of escapades demonstrating the adage ...
2                                                  A series
3                                                         A
4                                                    series
5         of escapades demonstrating the adage that what...
6                                                        of
7         escapades demonstrating the adage that what is...
8                                                 escapades
9         demonstrating the adage that what is good for ...
10                                  demonstrating the adage
11                                            demonstrating
12                                                the adage
13                                                      the
14                                                    adage
15                          that what is good for the goose
16                                      

Part3
构建神经网络，该部分代码是结合之前的一篇教程边学边改的
输入层L1暂定10000，单隐藏层L2暂定7，输出层L3暂定5，
L1->L2 sigmoid L2->L3 softmax

3.1前向传播

In [ ]:
def forward_propagation(X, parameters):

    W1 = parameters['W1']
    b1 = parameters['b1']
    W2 = parameters['W2']
    b2 = parameters['b2']    

    Z1 = np.dot(W1, X) + b1
    A1 = sigmoid(Z1)
    Z2 = np.dot(W2, A1) + b2
    A2 = softmax(Z2)    
    assert(A2.shape == (1, X.shape[1]))
 
 
    cache = {"Z1": Z1,                   
             "A1": A1,                   
             "Z2": Z2,                  
             "A2": A2}    
 
    return A2, cache

3.2计算损失函数
计算交叉熵函数

In [ ]:
def compute_cost(A2, Y, parameters):
    m = Y.num #Y中元素的个数
    logprobs = np.multiply(np.log(A2),Y) + np.multiply(np.log(1-A2), 1-Y)
    cost = -1/m * np.sum(logprobs)
    return cost

3.3 反向传播 （这部分应该还要继续改随机梯度下降）

In [ ]:
def backward_propagation(parameters, cache, X, Y):
    m = X.shape[1]    

    W1 = parameters['W1']
    W2 = parameters['W2']    

    A1 = cache['A1']
    A2 = cache['A2']    

    dZ2 = A2-Y
    dW2 = 1/m * np.dot(dZ2, A1.T)
    db2 = 1/m * np.sum(dZ2, axis=1, keepdims=True)
    dZ1 = np.dot(W2.T, dZ2)*(1-np.power(A1, 2))
    dW1 = 1/m * np.dot(dZ1, X.T)
    db1 = 1/m * np.sum(dZ1, axis=1, keepdims=True)
 
    grads = {"dW1": dW1,
             "db1": db1,                      
             "dW2": dW2,             
             "db2": db2}   
    return grads

3.4 权值更新

In [ ]:

def update_parameters(parameters, grads, learning_rate = 1.2):

    W1 = parameters['W1']
    b1 = parameters['b1']
    W2 = parameters['W2']
    b2 = parameters['b2']    

    dW1 = grads['dW1']
    db1 = grads['db1']
    dW2 = grads['dW2']
    db2 = grads['db2']    

    W1 -= dW1 * learning_rate
    b1 -= db1 * learning_rate
    W2 -= dW2 * learning_rate
    b2 -= db2 * learning_rate
 
    parameters = {"W1": W1, 
                  "b1": b1,            
                  "W2": W2,   
                  "b2": b2}    
    return parameters

part4 用试验集和测试集得出结果